In [1]:
import random
from collections import defaultdict

In [2]:
num_instruments = 100
names_ctr = ["name" + str(i) + "_ctr" for i in range(num_instruments)]
names_mmsc = ["name" + str(i) + "_mmsc" for i in range(num_instruments)]
descr = ["descr1", "descr2", "descr3"]
labels = ["key1, value1", "key2, value2", "key3, value3"]
instruments = ["ictr", "iudctr","ivrec","isobs","iudobs","ivobs","fctr", "fudctr","fvrec","fsobs","fudobs","fvobs","dctr", "dudctr","dvrec","dsobs","dudobs","dvobs","sctr", "sudctr","svrec","ssobs","sudobs","svobs"]

answers = defaultdict(lambda: defaultdict(lambda: [0,0,0,0]))

In [3]:
f = open("PrometheusDataFirst.csv", "w")

for i in range(100000):
    instr = random.choice(instruments)
    val = random.randint(-50,50)
    if (instr == "ictr" or instr == "isobs" or instr == "sctr" or instr == "ssobs" or instr == "dctr" or instr == "dsobs" or instr == "fctr" or instr == "fsobs"):
        val = abs(val)

    # TODO: might not need names + descriptions, will circle back after making load generator
    nam = ""
    if (instr == "ivrec" or instr == "ivobs" or instr == "svrec" or instr == "svobs" or instr == "dvrec" or instr == "dvobs" or instr == "fvrec" or instr == "fvobs"):
        nam = names_mmsc[random.randint(0, num_instruments-1)]
    else:
        nam = names_ctr[random.randint(0, num_instruments-1)]
    des = descr[random.randint(0, 2)]
    
    lab = random.sample(labels,random.randint(1,3))
    ndl = str(nam) + ", " + str(des) + ", " + str(lab)
    ndl = ndl.replace("[","").replace("]","").replace("'","")
    
    lab_str = "{" + ":".join(lab[0].split(', ')) + "}"
    for i in range(1, len(lab)):
        lab_str = lab_str + ", {" + ":".join(lab[i].split(', ')) + "}"
        
    ndl_frmt = str(nam) + ", " + str(des) + ", " + lab_str
        
    
    # mmsc aggregations
    if (instr == "ivrec" or instr == "ivobs" or instr == "svrec" or instr == "svobs" or instr == "dvrec" or instr == "dvobs" or instr == "fvrec" or instr == "fvobs"):
        answers[instr][ndl_frmt][0] = val if val < answers[instr][ndl_frmt][0] else answers[instr][ndl_frmt][0]
        answers[instr][ndl_frmt][1] = val if val > answers[instr][ndl_frmt][1] else answers[instr][ndl_frmt][1]
        answers[instr][ndl_frmt][2] += val
        answers[instr][ndl_frmt][3] += 1
    
    # counter aggregations
    else:
        answers[instr][ndl_frmt][0] += val
        answers[instr][ndl_frmt][1] += 1
#     print(instr+','+str(val)+','+"\""+ndl+"\"\n")
    f.write(instr+','+str(val)+','+"\""+ndl+"\"\n")
    
f.close()

In [5]:
fa = open("PrometheusAnswersFirst.csv", "w")

for instr in answers:
    for ndl in answers[instr]:
        
        # these instruments map to a MinMaxSumCount Aggregator in OTel, which map to a Gauge in Prometheus
        # the average value, sum/count, is used as the "last" value a Gauge in Prometheus keeps track of
        if (instr == "ivrec" or instr == "ivobs" or instr == "svrec" or instr == "svobs" or instr == "dvrec" or instr == "dvobs" or instr == "fvrec" or instr == "fvobs"):
            avg = answers[instr][ndl][2]/answers[instr][ndl][3]
#             print("gauge|{}|{}".format(str(avg),ndl))
            fa.write('gauge|'+str(avg)+'|'+ndl+'\n')
            
        # these instruments map to a Counter Aggregator in OTel, which map to a Counter in Prometheus
        # the sum is tracked by the Prometheus Counter so we validate only the sum, not count of updates, here
        else:
#             print("counter|{}|{}".format(str(answers[instr][ndl][0]),ndl))
            fa.write('counter|'+str(answers[instr][ndl][0])+'|'+ndl+'\n')
            
fa.close()